In [1]:
import gc
import sys
from statistics import mean
import time
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import itertools
from pathlib import Path
from scipy import special
import matplotlib.pyplot as plt
import datetime

import utils
import models
import learning_utils
from configurations import args_parser, arguments



In [2]:
start_time = time.time()
args = arguments()
#boardio is for the the tensorboardx prensation and textio is for written documentation
boardio, textio, best_val_acc, path_best_model, last_model_path = utils.initializations(args)
textio.cprint(str(args) if args.__class__.__name__ == 'Namespace' else str(vars(args)))

#mnist_train_data, mnist_test_loader  = utils.data(args)





{'exp_name': 'random selection', 'eval': False, 'data': 'cifar10', 'norm_std': 0.5, 'norm_mean': 0.5, 'train_batch_size': 20, 'test_batch_size': 1000, 'model': 'cnn5', 'num_users': 1, 'num_users_per_round': 1, 'local_epochs': 1, 'local_iterations': 100, 'global_epochs': 200, 'tau_min': 0.05, 'privacy_noise': 'laplace', 'epsilon': 4, 'optimizer': 'sgd', 'lr': 0.01, 'momentum': 0.5, 'lr_scheduler': True, 'device': 'cpu', 'seed': 0, 'zeta_coeff': 1.5, 'alpha': 1, 'beta': 2, 'gamma': 1, 'max_seconds': 200, 'method_choosing_users': 'random', 'data_truncation': None, 'choosing_users_verbose': False}


In [3]:
# data
train_data, test_loader = utils.data(args)
#input in the CNNs is the number of channels and in linear models is the size of the flatten pictures
input, output, train_data, val_loader = utils.data_split(train_data, len(test_loader.dataset), args)

# model
if args.model == 'mlp':
    global_model = models.FC2Layer(input, output)
elif args.model == 'cnn2':
    global_model = models.CNN2Layer(input, output, args.data)
elif args.model == 'cnn3':
    if args.data == 'mnist':
        raise ValueError('CNN3 is not supported for MNIST')
    global_model = models.CNN3Layer()
elif args.model == 'cnn5':
    if args.data == 'mnist':
        raise ValueError('CNN3 is not supported for MNIST')
    global_model = models.CNN5Layer(input, output)
elif args.model == 'linear':
    global_model = models.Linear(input, output)



textio.cprint(str(summary(global_model)).encode('utf-8', errors='ignore').decode('utf-8', errors='ignore'))
global_model.to(args.device)

train_criterion = torch.nn.CrossEntropyLoss(reduction='mean')
test_criterion = torch.nn.CrossEntropyLoss(reduction='sum')

local_models = utils.federated_setup(global_model, train_data, args, i_i_d=True)
utils.update_data_equility_partititon(local_models, args)



Files already downloaded and verified
Layer (type:depth-idx)                   Param #
CNN5Layer                                --
├─Conv2d: 1-1                            896
├─Conv2d: 1-2                            9,248
├─Conv2d: 1-3                            18,496
├─Conv2d: 1-4                            36,928
├─Conv2d: 1-5                            131,200
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            8,256
├─Linear: 1-8                            2,080
├─Linear: 1-9                            330
├─Dropout: 1-10                          --
├─Dropout: 1-11                          --
├─Dropout: 1-12                          --
├─BatchNorm2d: 1-13                      64
├─BatchNorm2d: 1-14                      128
├─BatchNorm2d: 1-15                      128
Total params: 207,754
Trainable params: 207,754
Non-trainable params: 0


In [4]:


choices_table = np.zeros((args.global_epochs, args.num_users))
num_of_obs_arr = np.zeros((1,args.num_users))
train_loss_list = []
val_acc_list = []
val_losses_list = []
global_epochs_time_list = []


time_counter = 0
for global_epoch in tqdm(range(1, args.global_epochs+1)):
    """Part 1: Choosing Users"""
    for usr_idx in range(args.num_users):
        local_models[usr_idx].update_g(global_epoch)
        local_models[usr_idx].update_ucb(global_epoch)
    
    if args.choosing_users_verbose:
        print(f"iteration{global_epoch}")
    a=time.time()
    rounds_choise = utils.choose_users(local_models, args, method=args.method_choosing_users, privacy=False)
    #print(f"choose_users took {time.time()-a}")
    
    #choices_table[global_epoch-1, rounds_choise] = 1
    num_of_obs_arr[0,rounds_choise] += 1
    for usr_idx in rounds_choise:
        local_models[usr_idx].update_emp_avg()
        local_models[usr_idx].update_privacy_violation_and_reward()
        local_models[usr_idx].increase_num_of_obs()
        if args.choosing_users_verbose:
            print(f"user {usr_idx}, g: {local_models[usr_idx].g},curr_delay = {local_models[usr_idx].last_access_time}, ucb: {local_models[usr_idx].ucb} num_of_obs: {local_models[usr_idx].num_of_obs}")
    
    max_delay = max([local_models[i].last_access_time for i in rounds_choise])
    if args.choosing_users_verbose:
        print(f"max_delay = {max_delay}")
    
    
    """Part 2: Training"""
    learning_utils.distribute_model(local_models, global_model)
    users_avg_loss_over_local_epochs = []

    for user_idx in rounds_choise:
        user_loss = []
        for local_epoch in range(args.local_epochs):
            user = local_models[user_idx]
            train_loss = learning_utils.train_one_epoch(user, train_criterion, args)
            if args.lr_scheduler:
                user.scheduler.step(train_loss)
            user_loss.append(train_loss)
        users_avg_loss_over_local_epochs.append(mean(user_loss))
    
    avg_loss_over_chosen_users_curr_global_epoch = mean(users_avg_loss_over_local_epochs)
    train_loss_list.append(avg_loss_over_chosen_users_curr_global_epoch)

    #we return deltha_theta only for checking can be removed and fix the return value of the function later on to None
    deltha_theta = learning_utils.Fed_avg_models(local_models, global_model, rounds_choise)
    val_acc, val_loss = learning_utils.test(test_loader, global_model, test_criterion, args)
    val_acc_list.append(val_acc) ; val_losses_list.append(val_loss)
    

    # boardio.add_scalars("Losses over time in seconds", {"train_loss":avg_loss_over_chosen_users_curr_global_epoch,
    #                                     "val_loss": val_loss}, time.time()-start_time)
    # boardio.add_scalar('Val Accuracy', val_acc, time.time()-start_time)


    time_counter += max_delay
    print((f"global epoch {global_epoch} has been done artifficialy in {max_delay:.2f} secs, the total time by now is {time_counter:.2f} \n with avg train loss {avg_loss_over_chosen_users_curr_global_epoch:.3f}, avg val loss {val_loss:.3f}, avg val acc {val_acc:.2f}%"))
    global_epochs_time_list.append(time_counter)
    gc.collect()


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({"model's state dict":global_model.state_dict(),
                    "train_loss_list": train_loss_list,
                    "val_acc_list": val_acc_list,
                    "val_losses_list": val_losses_list,
                    "global_epochs_time_list": global_epochs_time_list,
                    "num_of_users": args.num_users,
                    "num_of_users_per_round": args.num_users_per_round}
                    , path_best_model)
    
    
    torch.save({"model's state dict":global_model.state_dict(),
                "train_loss_list": train_loss_list,
                "val_acc_list": val_acc_list,
                "val_losses_list": val_losses_list,
                "global_epochs_time_list": global_epochs_time_list,
                "num_of_obs_arr": num_of_obs_arr.reshape(-1),
                "global_epoch": global_epoch,
                "num_of_users": args.num_users,
                "num_of_users_per_round": args.num_users_per_round}
                , last_model_path)

    if time_counter > args.max_seconds:
        break

users_idxs = tuple([str(x) for x in range(1,args.num_users+1)])
fig, ax = plt.subplots()
ax.bar(users_idxs, num_of_obs_arr.reshape(-1), width = 0.4)
ax.set_title("Number of times each user was chosen")
ax.set_ylabel("number of times")
ax.set_xlabel("user index")
#boardio.add_figure("Number of times each user was chosen", fig, global_epoch)
plt.savefig(last_model_path.parent / "Number of times each user was chosen.png")

#choices_table = choices_table.cumsum(axis=0)


  0%|          | 1/200 [00:19<1:03:32, 19.16s/it]

global epoch 1 has been done artifficialy in 0.43 secs, the total time by now is 0.43 
 with avg train loss 2.289, avg val loss 2.249, avg val acc 18.23%


  1%|          | 2/200 [00:38<1:03:10, 19.14s/it]

global epoch 2 has been done artifficialy in 1.52 secs, the total time by now is 1.95 
 with avg train loss 2.214, avg val loss 2.095, avg val acc 26.45%


  2%|▏         | 3/200 [00:56<1:01:57, 18.87s/it]

...

global epoch 81 has been done artifficialy in 1.60 secs, the total time by now is 72.32 
 with avg train loss 0.847, avg val loss 0.871, avg val acc 69.47%


 41%|████      | 82/200 [26:13<38:40, 19.67s/it]

global epoch 82 has been done artifficialy in 0.69 secs, the total time by now is 73.01 
 with avg train loss 0.830, avg val loss 0.835, avg val acc 71.04%


 42%|████▏     | 83/200 [26:33<38:45, 19.88s/it]

global epoch 83 has been done artifficialy in 0.81 secs, the total time by now is 73.82 
 with avg train loss 0.816, avg val loss 0.843, avg val acc 70.87%


In [ ]:
%tensorboard --logdir=runs